In [1]:
import torch
import gc
torch.cuda.empty_cache()
gc.collect()


0

In [2]:
from setup import setup_src_path
print(setup_src_path())
import data.processed as processed
import config.config as config
import utils.setup as setup
import utils.functions as fn
from importlib import reload



['/home/guest/Desktop/projects/third-experiments/domain_adaptation_project/mlm', '/usr/lib/python38.zip', '/usr/lib/python3.8', '/usr/lib/python3.8/lib-dynload', '', '/home/guest/.cache/pypoetry/virtualenvs/third-experments-xuKQSur9-py3.8/lib/python3.8/site-packages', '/home/guest/Desktop/projects/third-experiments/domain_adaptation_project/modules']


2024-06-05 04:22:32.523183: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-05 04:22:32.719855: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-05 04:22:33.592577: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
from datasets import load_from_disk,concatenate_datasets


adapter_name="mlm_union_all_TE"
dataset = load_from_disk(f"{config.Config.DATASETS_SAVE_PATH}/datasets")



In [4]:
reload(config)

    
filtered_target = dataset['train'].filter(lambda example: example['genre'] == "telephone").shuffle(seed=42)
#filtered_target = shuffled_filtered_target.train_test_split(test_size=0.1)


filtered_test_target = dataset['validation_matched'].filter(lambda example: example['genre'] == "telephone")
train_target = filtered_target
test_target = filtered_test_target
train_target[2]
 

{'promptID': 8103,
 'pairID': '8103e',
 'premise': "but i like dogs and my husband like cats so we haven't reached a real agreement on that yet  if we get a place where we can have both it'll be great but until then",
 'premise_binary_parse': "( but ( ( i ( like ( ( dogs and ) ( ( my husband ) ( like cats ) ) ) ) ) ( so ( we ( ( have n't ) ( reached ( ( a ( real agreement ) ) ( ( on that ) ( ( yet ( if ( we ( get ( ( a place ) ( where ( we ( can ( have both ) ) ) ) ) ) ) ) ) ( it ( 'll ( be ( great ( but ( until then ) ) ) ) ) ) ) ) ) ) ) ) ) ) )",
 'premise_parse': "(ROOT (FRAG (CC but) (ADVP (RB i) (PP (IN like) (NP (NP (NNS dogs)) (CC and) (NP (NP (PRP$ my) (NN husband)) (PP (IN like) (NP (NNS cats))))))) (SBAR (IN so) (S (NP (PRP we)) (VP (VBP have) (RB n't) (VP (VBN reached) (NP (NP (DT a) (JJ real) (NN agreement)) (SBAR (WHPP (IN on) (WHNP (WDT that))) (S (SBAR (RB yet) (IN if) (S (NP (PRP we)) (VP (VBP get) (NP (NP (DT a) (NN place)) (SBAR (WHADVP (WRB where)) (S (NP (PRP we)) (

In [5]:
from adapters import AutoAdapterModel,init
from tqdm import tqdm

from transformers import AutoConfig, AutoModelForMaskedLM

mdlcfg = AutoConfig.from_pretrained(
    config.Config.MODEL_NAME,
 
)
model = AutoModelForMaskedLM.from_pretrained(
    config.Config.MODEL_NAME,
)
init(model)
reload(fn)
fn.print_trainable_parameters(model)

/home/guest/.cache/pypoetry/virtualenvs/third-experments-xuKQSur9-py3.8/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertFor

trainable params: 109514298 || all params: 109514298 || trainable%: 100.0


In [6]:
from adapters import ConfigUnion, LoRAConfig, PrefixTuningConfig, SeqBnConfig,SeqBnInvConfig,AdapterConfig,LoRAConfig

union_config = ConfigUnion(
    PrefixTuningConfig(prefix_length=10),
    LoRAConfig(r=8, alpha=16),

    SeqBnInvConfig(reduction_factor=2),
)
model.add_adapter(adapter_name, config=union_config)

model.train_adapter([adapter_name])
model.active_adapters = adapter_name
print(model.adapter_summary())
fn.print_trainable_parameters(model)

Name                     Architecture         #Param      %Param  Active   Train
--------------------------------------------------------------------------------
mlm_union_all_TE         union            17,539,712      16.107       1       1
--------------------------------------------------------------------------------
Full model                               108,891,648     100.000               0
trainable params: 18131840 || all params: 127054010 || trainable%: 14.2709702747674


In [7]:
from transformers import AutoTokenizer

reload(processed)
tokenizer = AutoTokenizer.from_pretrained(config.Config.TOKENIZER_NAME)

tokenized_dataset= processed.tokenize_dataset(train_target,tokenizer)
tokenized_dataset

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids'],
    num_rows: 83348
})

In [8]:
chunk_size = 128
# Slicing produces a list of lists for each feature
tokenized_samples = tokenized_dataset[444:470]

results = fn.group_texts(tokenized_samples, chunk_size)
for chunk in results["labels"]:
    print(f"'>>> Chunk length: {len(chunk)}'")

'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'


In [9]:
# we group texts and chunk them
lm_datasets = tokenized_dataset.map(fn.group_texts, batched=True,fn_kwargs={'chunk_size': chunk_size})
lm_datasets

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids', 'labels'],
    num_rows: 27551
})

In [10]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)
samples = [lm_datasets[i] for i in range(2)]
for sample in samples:
    _ = sample.pop("word_ids")
dd=data_collator(samples)
for chunk in dd["input_ids"]:
    print(f"\n'>>> {tokenizer.decode(chunk)}'")


'>>> [CLS] i mean i was absolutely [MASK]abbergasted [MASK] what she [MASK] would that she was paid true [MASK] was [MASK] you know but [MASK] it's the principle of course if she got her master's degree it'd all be different but uh [SEP] i [MASK] she [MASK] much more money. [SEP] [CLS] yeah [MASK] you know and it [MASK] are many of them there'[MASK] no doubt about it [SEP] there [MASK] countless numbers of them, that is for sure. [SEP] [CLS] but i [MASK] [MASK] and my [MASK] like cats so [MASK] haven't reached a real agreement on that yet if we get a place where we can have both it ''

'>>> ll be great but until then [SEP] [MASK] husband and [MASK] can't agree on a pet. [SEP] [CLS] yeah [MASK] me [MASK] [MASK] [MASK] me my [MASK] husband's putting two of us through isn't he lucky [SEP] my husband is lucky [MASK] he is putting [MASK] of us through. [SEP] [CLS] never breaks down [MASK] the [MASK]yrofoam [SEP] never [MASK] the styrofoam [MASK] [SEP] [CLS] i haven't seen that [MASK] i and

In [11]:

downsampled_dataset = lm_datasets.train_test_split(
    test_size=0.1, seed=42
)
downsampled_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 24795
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 2756
    })
})

In [12]:

reload(fn)
trainer = fn.train_mlm_model(model,adapter_name,data_collator,tokenizer, downsampled_dataset['train'],downsampled_dataset['test'])

/home/guest/.cache/pypoetry/virtualenvs/third-experments-xuKQSur9-py3.8/lib/python3.8/site-packages/accelerate/accelerator.py:446: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [13]:
import math

eval_results = trainer.evaluate()

print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

  0%|          | 0/87 [00:00<?, ?it/s]

>>> Perplexity: 23.07


In [14]:
trainer.train()

  0%|          | 0/15500 [00:00<?, ?it/s]

{'loss': 2.3204, 'grad_norm': 1.7871910333633423, 'learning_rate': 9.817333333333334e-05, 'epoch': 1.0}


  0%|          | 0/87 [00:00<?, ?it/s]

{'eval_loss': 1.9531549215316772, 'eval_runtime': 9.6013, 'eval_samples_per_second': 287.045, 'eval_steps_per_second': 9.061, 'epoch': 1.0}
{'loss': 1.9921, 'grad_norm': 1.5396252870559692, 'learning_rate': 9.301333333333335e-05, 'epoch': 2.0}


  0%|          | 0/87 [00:00<?, ?it/s]

...

  0%|          | 0/87 [00:00<?, ?it/s]

{'eval_loss': 1.6334829330444336, 'eval_runtime': 10.3275, 'eval_samples_per_second': 266.861, 'eval_steps_per_second': 8.424, 'epoch': 18.0}


Overwriting existing adapter 'mlm_union_all_TE'.


{'train_runtime': 3571.2574, 'train_samples_per_second': 138.859, 'train_steps_per_second': 4.34, 'train_loss': 1.8053313615587023, 'epoch': 18.0}


TrainOutput(global_step=13950, training_loss=1.8053313615587023, metrics={'train_runtime': 3571.2574, 'train_samples_per_second': 138.859, 'train_steps_per_second': 4.34, 'train_loss': 1.8053313615587023, 'epoch': 18.0})

In [15]:
import math

eval_results = trainer.evaluate()

print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

  0%|          | 0/87 [00:00<?, ?it/s]

>>> Perplexity: 5.06


In [16]:
import config.config as config

trainer.model.save_adapter(f"{config.Config.ADAPTER_SAVE_PATH}/{adapter_name}", adapter_name)


: 

#### This is a report of mlm for Slate domain using whole genre as target and then splitt after tokenization with seed 42. I used computer metrix for preplexity and early stopping 
The results before:
>>> Perplexity: 14.65

The results after:
>>> Perplexity: 5.06